In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from utils import save_config

pd.set_option('display.max_rows', 150)
ID = 1

In [2]:
rand_seed = 42

In [3]:
prj_folder = Path('.').resolve()
prj_folder

WindowsPath('C:/Users/pavel/OneDrive/Desktop/it_cup')

In [4]:
data = pd.read_csv(prj_folder/'2. Cup_IT_2022_Датасет_Data_Science.csv')

In [5]:
X = data.iloc[:, 2:]
y = data.iloc[:, 1]

X = SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(X)

X_train, X_val, y_train, y_val = train_test_split(X, y, shuffle=False, train_size=0.3, random_state=42)

In [6]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, recall_score, precision_score
precision_score(y, np.zeros_like(y))

C:\Users\pavel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.0

In [7]:
rf_config = {
    #'n_estimators': [200, 500],
    #'max_features': ['auto', 'sqrt', 'log2'],
    #'max_depth' : [4,5,6,7,8],
    #'criterion' :['gini', 'entropy'],
    'class_weight': [{0: 0.09, 1: 0.91}]
}

gb_config = {
    'learning_rate': [0.1],
    'n_estimators': [200]
}

scoring = ['accuracy', 'f1', 'roc_auc', 'precision', 'recall']

In [8]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

clf = RandomForestClassifier(random_state=rand_seed, n_jobs=8)
#clf = GradientBoostingClassifier(random_state=rand_seed)

In [9]:
CUR_CONFIG = rf_config

save_config(ID, CUR_CONFIG, clf)
ID += 1

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=rand_seed)
validator = GridSearchCV(clf, param_grid=CUR_CONFIG, scoring=scoring, n_jobs=4, cv=cv, refit='f1', verbose=2)

In [10]:
validator.fit(X_train, y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=42, shuffle=True),
             estimator=RandomForestClassifier(n_jobs=8, random_state=42),
             n_jobs=4, param_grid={'class_weight': [{0: 0.09, 1: 0.91}]},
             refit='f1',
             scoring=['accuracy', 'f1', 'roc_auc', 'precision', 'recall'],
             verbose=2)

In [11]:
print(validator.best_estimator_)
print(validator.best_params_)
print(validator.best_score_)
print(validator.cv_results_)


RandomForestClassifier(class_weight={0: 0.09, 1: 0.91}, n_jobs=8,
                       random_state=42)
{'class_weight': {0: 0.09, 1: 0.91}}
0.008910703284189306
{'mean_fit_time': array([14.9789416]), 'std_fit_time': array([0.17603397]), 'mean_score_time': array([0.36668706]), 'std_score_time': array([0.01846776]), 'param_class_weight': masked_array(data=[{0: 0.09, 1: 0.91}],
             mask=[False],
       fill_value='?',
            dtype=object), 'params': [{'class_weight': {0: 0.09, 1: 0.91}}], 'split0_test_accuracy': array([0.87390789]), 'split1_test_accuracy': array([0.87346365]), 'split2_test_accuracy': array([0.87383385]), 'mean_test_accuracy': array([0.87373513]), 'std_test_accuracy': array([0.00019433]), 'rank_test_accuracy': array([1]), 'split0_test_f1': array([0.00930774]), 'split1_test_f1': array([0.00927536]), 'split2_test_f1': array([0.00814901]), 'mean_test_f1': array([0.0089107]), 'std_test_f1': array([0.00053876]), 'rank_test_f1': array([1]), 'split0_test_roc_auc'

In [12]:
from utils import fine_print
fine_print(validator.cv_results_)

m_acc:     [0.87373513]
m_f1:      [0.0089107]
m_rauc:    [0.67519525]
m_prec:    [0.35235525]
m_recall:  [0.00451338]


In [13]:
from sklearn.metrics import confusion_matrix
preds = validator.predict(X_val)
confusion_matrix(y_val, preds)

array([[87160,    61],
       [ 7290,    32]], dtype=int64)

In [16]:
print(accuracy_score(y_val, preds))
print(f1_score(y_val, preds))
print(roc_auc_score(y_val, preds))
print(precision_score(y_val, preds))
print(recall_score(y_val, preds))

0.9222470198745545
0.008631153068105192
0.5018355088731031
0.34408602150537637
0.004370390603660202
